In [7]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense,Dropout,Activation
from keras.optimizers import Adam
from keras.utils import np_utils
import math
import matplotlib.pyplot as plt

In [46]:
df_all = pd.read_csv('../rnn_data/all/2013_2018_price.csv',encoding='utf-8')
# df_all = pd.read_csv('2013_2018_price.csv',encoding='utf-8')
df_c = df_all.groupby('證券代號').get_group('2317')

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
def batch_data(df_c,time_steps,seperate_rate):
    del df_c['Unnamed: 0']
    del df_c['最後揭示買價']
    del df_c['最後揭示買量']
    del df_c['最後揭示賣價']
    del df_c['最後揭示賣量']
    del df_c['證券代號']
    del df_c['本益比']
    del df_c['成交金額']
    del df_c['證券名稱']
    del df_c['Unnamed: 0.1']
    empty_li = df_c[df_c['漲跌(+/-)']=='X'].index.values
    for i in empty_li:
        df_c = df_c.drop(i)
    empty_li = df_c[df_c['漲跌(+/-)']==' '].index.values
    for i in empty_li:
        df_c = df_c.drop(i)
    empty_li = df_c[df_c['收盤價']=='X'].index.values
    for i in empty_li:
        df_c = df_c.drop(i)
    empty_li = df_c[df_c['收盤價']==' '].index.values
    for i in empty_li:
        df_c = df_c.drop(i)
    df_c['成交筆數']=df_c['成交筆數'].str.replace(',','')
    df_c['成交股數']=df_c['成交股數'].str.replace(',','')
    df_c.成交筆數 = df_c.成交筆數.astype(float)
    df_c.成交股數 = df_c.成交股數.astype(float)
    df_c.收盤價 = df_c.收盤價.astype(float)
    df_c.最低價 = df_c.最低價.astype(float)
    df_c.最高價 = df_c.最高價.astype(float)
    df_c.開盤價 = df_c.開盤價.astype(float)
    df_c.漲跌價差 = df_c.漲跌價差.astype(float)
    df_c['return'] = np.where(df_c['漲跌(+/-)'] !='+',df_c['漲跌價差']*-1, df_c['漲跌價差'])
    df_c['return']=df_c['return'].shift(-1)
#     df_c['label'] = 100*(df_c['return']+df_c['return'].shift(-4)+df_c['return'].shift(-3)+df_c['return'].shift(-2)+df_c['return'].shift(-1))/df_c['收盤價']
#     df_c['label'] = df_c['label'].apply(lambda x:np.where(x>=3,3,np.where(x>-3,x,-3)))
    df_c['label'] = ((df_c['收盤價']-df_c['收盤價'].shift(-5))/df_c['收盤價']).apply(lambda x:np.where(x>=0.2,0.2,np.where(x>-0.2,x,-0.2)))
    del df_c['漲跌(+/-)']
    del df_c['漲跌價差']
    del df_c['return']
    d= df_c['日期']
    del df_c['日期']
    min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
    for i in range(len(df_c.columns)-1):
        df_c[df_c.columns[i]] =min_max_scaler.fit_transform(np.array(df_c[df_c.columns[i]]).reshape(-1,1))
    Y_batch = df_c['label']
    del df_c['label']
    X_batch = df_c
    X_batch = np.array(X_batch)
    Y_batch = np.array(Y_batch)
    X_batch_re=[]
    Y_batch_re=[]
    for i in range(len(X_batch)):
        X_batch_re.append(X_batch[i:time_steps+i])
        Y_batch_re.append(Y_batch[i])
    n=len(X_batch_re)
    batch_n = math.floor(n/BATCH_SIZE)
    drop_n = n-batch_n*BATCH_SIZE
    X_batch_re=X_batch_re[drop_n:]
    Y_batch_re=Y_batch_re[drop_n:]
    train_size=math.ceil(len(X_batch_re)*seperate_rate)
    X_batch_train=X_batch_re[:train_size]
    Y_batch_train=Y_batch_re[:train_size]
    X_batch_test=X_batch_re[train_size:]
    Y_batch_test=Y_batch_re[train_size:]
    return X_batch_train,Y_batch_train,X_batch_test,Y_batch_test

In [48]:
time_steps=30
seperate_rate=0.9
BATCH_SIZE=5

In [49]:
X_batch_train,Y_batch_train,X_batch_test,Y_batch_test=batch_data(df_c,time_steps,seperate_rate)

In [50]:
TIME_STEPS = time_steps
INPUT_SIZE = 6
OUTPUT_SIZE=32
LR = 0.0005

In [51]:
model =Sequential()
model.add(LSTM(
    batch_input_shape=(1,TIME_STEPS, INPUT_SIZE),
    output_dim=32,
    return_sequences=False,
    stateful=True,
))
model.add(Dropout(0.2))
model.add(Dense(8))
model.add(Dense(1))
adam = Adam(LR)
model.compile(optimizer=adam,
              loss='mse')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(batch_input_shape=(1, 30, 6), return_sequences=False, stateful=True, units=32)`
  


In [52]:
X_batch_train=np.array(X_batch_train)
Y_batch_train = np.array(Y_batch_train)

In [53]:
Y_batch_train[0]

0.0056116722783389455

In [56]:
Y_batch_train[0]

0.0056116722783389455

In [ ]:
epoch = 1001
cl = []
for epoch in range(epoch):
    for i in range((X_batch_train).shape[0]):
        
        X = X_batch_train[i].reshape(1,30,6)

        Y = Y_batch_train[i].reshape(1,1)

        cost = model.train_on_batch(X,Y)
        pred = model.predict(X, BATCH_SIZE)
        cl.append(cost)
    if epoch % 100  == 0:
        print('cl',np.average(cl))
        print('train cost: ', cost,'epoch: ',epoch)
        cl=[]

cl 0.005157717
train cost:  0.001425131 epoch:  0
cl 0.0008801822
train cost:  1.8651245e-06 epoch:  100
cl 0.00070746284
train cost:  1.6085645e-05 epoch:  200
cl 0.00051978184
train cost:  5.195356e-06 epoch:  300
cl 0.00041636257
train cost:  1.736643e-05 epoch:  400


In [132]:
drop_location = 0
for i in range(X_batch_test_time_b.shape[0]):
    if X_batch_test_time_b[i].shape[0]==0:
        drop_location = i
        print(X_batch_test_time_b[i].shape[0])
        print(i)
        break
X_batch_test_time_b=X_batch_test_time_b[0:drop_location]
Y_batch_test_time_b=Y_batch_test_time_b[0:drop_location]

0
22


In [133]:
yhat=[]
for i in range(X_batch_test_time_b.shape[0]):
    print(X_batch_test_time_b[i].shape)
    yhat.append(model.predict_on_batch(X_batch_test_time_b[i]))

(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)
(5, 5, 6)


In [136]:
yhat =np.array(yhat).flatten()

In [137]:
yhat.shape

(550, 1)

In [143]:
Y_batch_test_time_b

array([[[0.09251004],
        [0.09761282],
        [0.91461101],
        [0.91395793],
        [0.92110454]],

       [[0.89320388],
        [0.08457924],
        [0.08315001],
        [0.90512334],
        [0.90439771]],

       [[0.93096647],
        [0.91262136],
        [0.11356895],
        [0.13556097],
        [0.89563567]],

       [[0.90439771],
        [0.92110454],
        [0.88349515],
        [0.15220233],
        [0.15906118]],

       [[0.87666034],
        [0.86615679],
        [0.89151874],
        [0.86407767],
        [0.28384273]]])

In [55]:
plt.plot(yhat,color='blue')
plt.plot(Y_batch_test_time_b,color='red')
plt.show()